## Bilgisayar Mimarisi
### Assembly Register Simülatörü

In [169]:
class CPU:
  def __init__(self, komutlar, pc=1, memory={}):
    self.komutlar = komutlar
    self.pc = pc
    self.mar = None
    self.mbr = None
    self.ir = None
    self.acc = None
    self.al = None
    self.bl = None
    self.memory = memory

  def yazdir(self):
    print(f'PC: {self.pc} '\
          f'MAR: {self.mar} '\
          f'MBR: {self.mbr} '\
          f'IR: {self.ir} '\
          f'ACC: {self.acc} '\
          f'AL: {self.al} '\
          f'BL: {self.bl}')
  
  def akis1(self):
      self.mar = self.pc
      self.mbr = f'I{self.pc}'
      self.ir = f'I{self.pc}'
      self.pc += 1
      self.yazdir()
    
  def calistir(self):
    self.yazdir()
    for komut in self.komutlar:
      komut_islem, komut_veri = komut.split(" ")

      if komut_islem == "LOAD":
        self.akis1()
        self.acc = memory.get(komut_veri)
        self.mar = komut_veri
        self.yazdir()
      elif komut_islem == "MOV":
        self.akis1()
        hedef, kaynak = komut_veri.split(",")
        if hedef == "AL":
          if kaynak == "BL":
            self.al = self.bl
          else:
            if kaynak[0] == "#":
              self.al = int(kaynak[1:])
            elif kaynak[0] == "@":
              kaynak = kaynak[1:]
              self.mar = kaynak
              self.mbr = self.memory.get(kaynak)
              self.yazdir()
              self.mar = self.mbr
              self.mbr = self.memory.get(self.mbr)
              self.al = self.mbr
            else:
              self.mar = kaynak
              self.mbr = self.memory.get(kaynak)
              self.al = self.memory.get(kaynak)
        if hedef == "BL":
          if kaynak == "AL":
            self.bl = self.al
          else:
            if kaynak[0] == "#":
              self.bl = int(kaynak[1:])
            elif kaynak[0] == "@":
              kaynak = kaynak[1:]
              self.mar = kaynak
              self.mbr = self.memory.get(kaynak)
              self.yazdir()
              self.mar = self.mbr
              self.mbr = self.memory.get(self.mbr)
              self.al = self.mbr
            else:
              self.mar = kaynak
              self.mbr = self.memory.get(kaynak)
              self.bl = self.memory.get(kaynak)
        self.yazdir()
      elif komut_islem == "ADD":
        self.akis1()
        hedef, kaynak = komut_veri.split(",")
        if (hedef == "AL" or hedef == "BL") and (kaynak == "AL" or kaynak == "BL"):
          self.acc = int(self.al) + int(self.bl)
        else:
          access_memory_hedef, access_memory_kaynak = False, False
          if hedef.isdigit():
            access_memory_hedef = True
          if kaynak.isdigit():
            access_memory_kaynak = True
          if hedef == "AL":
            hedef = self.al
          if hedef == "BL":
            hedef = self.bl
          if kaynak == "AL":
            kaynak = self.al
          if kaynak == "BL":
            kaynak = self.bl
          hedef = str(hedef)
          kaynak = str(kaynak)
          if hedef[0] == "#":
            hedef = hedef[1:]
          if kaynak[0] == "#":
            kaynak = kaynak[1:]
          if access_memory_hedef:
            self.mar = hedef
            self.mbr = self.memory.get(self.mar)
            hedef = self.mbr
          # aynı anda ikisi de olunca düzgün çalışmayabilir
          if access_memory_kaynak:
            self.mar = kaynak
            self.mbr = self.memory.get(self.mar)
            kaynak = self.mbr
          self.acc = int(kaynak) + int(hedef)
        self.yazdir()
      elif komut_islem == "STO":
        self.akis1()
        if komut_veri[0] == "@":
          komut_veri = komut_veri[1:]
          self.mar = komut_veri
          self.mbr = self.memory.get(komut_veri)
          self.yazdir()
          self.mar = self.mbr
          self.memory[self.mar] = self.acc
        else:
          self.mar = komut_veri
          self.memory[komut_veri] = self.acc
        self.yazdir()


    self.akis1()

In [170]:
memory = {"10021" : 2013, 
          "10020": "10022", 
          "10022": "10025", 
          "10025": 1453}
komutlar = ["LOAD 10021", "MOV AL,10025", "MOV BL,AL", "MOV AL,@10020", "ADD AL,BL", "STO 10022"]

islemci1 = CPU(komutlar, pc=10010, memory=memory)
islemci1.calistir()

PC: 10010 MAR: None MBR: None IR: None ACC: None AL: None BL: None
PC: 10011 MAR: 10010 MBR: I10010 IR: I10010 ACC: None AL: None BL: None
PC: 10011 MAR: 10021 MBR: I10010 IR: I10010 ACC: 2013 AL: None BL: None
PC: 10012 MAR: 10011 MBR: I10011 IR: I10011 ACC: 2013 AL: None BL: None
PC: 10012 MAR: 10025 MBR: 1453 IR: I10011 ACC: 2013 AL: 1453 BL: None
PC: 10013 MAR: 10012 MBR: I10012 IR: I10012 ACC: 2013 AL: 1453 BL: None
PC: 10013 MAR: 10012 MBR: I10012 IR: I10012 ACC: 2013 AL: 1453 BL: 1453
PC: 10014 MAR: 10013 MBR: I10013 IR: I10013 ACC: 2013 AL: 1453 BL: 1453
PC: 10014 MAR: 10020 MBR: 10022 IR: I10013 ACC: 2013 AL: 1453 BL: 1453
PC: 10014 MAR: 10022 MBR: 10025 IR: I10013 ACC: 2013 AL: 10025 BL: 1453
PC: 10015 MAR: 10014 MBR: I10014 IR: I10014 ACC: 2013 AL: 10025 BL: 1453
PC: 10015 MAR: 10014 MBR: I10014 IR: I10014 ACC: 11478 AL: 10025 BL: 1453
PC: 10016 MAR: 10015 MBR: I10015 IR: I10015 ACC: 11478 AL: 10025 BL: 1453
PC: 10016 MAR: 10022 MBR: I10015 IR: I10015 ACC: 11478 AL: 10025 BL

In [171]:
memory = {"13": 23, "23": 13}
komutlar = ["MOV AL,13", "ADD AL,23", "STO 33"]

islemci1 = CPU(komutlar, memory=memory)
islemci1.calistir()

PC: 1 MAR: None MBR: None IR: None ACC: None AL: None BL: None
PC: 2 MAR: 1 MBR: I1 IR: I1 ACC: None AL: None BL: None
PC: 2 MAR: 13 MBR: 23 IR: I1 ACC: None AL: 23 BL: None
PC: 3 MAR: 2 MBR: I2 IR: I2 ACC: None AL: 23 BL: None
PC: 3 MAR: 23 MBR: 13 IR: I2 ACC: 36 AL: 23 BL: None
PC: 4 MAR: 3 MBR: I3 IR: I3 ACC: 36 AL: 23 BL: None
PC: 4 MAR: 33 MBR: I3 IR: I3 ACC: 36 AL: 23 BL: None
PC: 5 MAR: 4 MBR: I4 IR: I4 ACC: 36 AL: 23 BL: None


In [172]:
memory = {"13": 123, "23": 223}
komutlar = ["MOV AL,#23", "MOV BL,13", "ADD AL,BL"]

islemci1 = CPU(komutlar, memory=memory)
islemci1.calistir()

PC: 1 MAR: None MBR: None IR: None ACC: None AL: None BL: None
PC: 2 MAR: 1 MBR: I1 IR: I1 ACC: None AL: None BL: None
PC: 2 MAR: 1 MBR: I1 IR: I1 ACC: None AL: 23 BL: None
PC: 3 MAR: 2 MBR: I2 IR: I2 ACC: None AL: 23 BL: None
PC: 3 MAR: 13 MBR: 123 IR: I2 ACC: None AL: 23 BL: 123
PC: 4 MAR: 3 MBR: I3 IR: I3 ACC: None AL: 23 BL: 123
PC: 4 MAR: 3 MBR: I3 IR: I3 ACC: 146 AL: 23 BL: 123
PC: 5 MAR: 4 MBR: I4 IR: I4 ACC: 146 AL: 23 BL: 123


In [173]:
memory = {"13": 33, "23": 123, "33": 333}
komutlar = ["MOV BL,33", "ADD 23,#23", "STO 13"]

islemci1 = CPU(komutlar, memory=memory)
islemci1.calistir()

PC: 1 MAR: None MBR: None IR: None ACC: None AL: None BL: None
PC: 2 MAR: 1 MBR: I1 IR: I1 ACC: None AL: None BL: None
PC: 2 MAR: 33 MBR: 333 IR: I1 ACC: None AL: None BL: 333
PC: 3 MAR: 2 MBR: I2 IR: I2 ACC: None AL: None BL: 333
PC: 3 MAR: 23 MBR: 123 IR: I2 ACC: 146 AL: None BL: 333
PC: 4 MAR: 3 MBR: I3 IR: I3 ACC: 146 AL: None BL: 333
PC: 4 MAR: 13 MBR: I3 IR: I3 ACC: 146 AL: None BL: 333
PC: 5 MAR: 4 MBR: I4 IR: I4 ACC: 146 AL: None BL: 333


In [174]:
memory = {"33021": "66044", "33023": "33021"}
komutlar = ["LOAD 33023", "MOV AL,33023", "MOV BL,#33023", "ADD AL,BL", "STO @33023"]

islemci1 = CPU(komutlar, memory=memory, pc=33011)
islemci1.calistir()

PC: 33011 MAR: None MBR: None IR: None ACC: None AL: None BL: None
PC: 33012 MAR: 33011 MBR: I33011 IR: I33011 ACC: None AL: None BL: None
PC: 33012 MAR: 33023 MBR: I33011 IR: I33011 ACC: 33021 AL: None BL: None
PC: 33013 MAR: 33012 MBR: I33012 IR: I33012 ACC: 33021 AL: None BL: None
PC: 33013 MAR: 33023 MBR: 33021 IR: I33012 ACC: 33021 AL: 33021 BL: None
PC: 33014 MAR: 33013 MBR: I33013 IR: I33013 ACC: 33021 AL: 33021 BL: None
PC: 33014 MAR: 33013 MBR: I33013 IR: I33013 ACC: 33021 AL: 33021 BL: 33023
PC: 33015 MAR: 33014 MBR: I33014 IR: I33014 ACC: 33021 AL: 33021 BL: 33023
PC: 33015 MAR: 33014 MBR: I33014 IR: I33014 ACC: 66044 AL: 33021 BL: 33023
PC: 33016 MAR: 33015 MBR: I33015 IR: I33015 ACC: 66044 AL: 33021 BL: 33023
PC: 33016 MAR: 33023 MBR: 33021 IR: I33015 ACC: 66044 AL: 33021 BL: 33023
PC: 33016 MAR: 33021 MBR: 33021 IR: I33015 ACC: 66044 AL: 33021 BL: 33023
PC: 33017 MAR: 33016 MBR: I33016 IR: I33016 ACC: 66044 AL: 33021 BL: 33023


In [175]:
memory = {"123": "10", "5": "123", "12":"314"}
komutlar = ["LOAD 123", "MOV AL,12", "STO @5"]

islemci1 = CPU(komutlar, memory=memory, pc=1)
islemci1.calistir()

PC: 1 MAR: None MBR: None IR: None ACC: None AL: None BL: None
PC: 2 MAR: 1 MBR: I1 IR: I1 ACC: None AL: None BL: None
PC: 2 MAR: 123 MBR: I1 IR: I1 ACC: 10 AL: None BL: None
PC: 3 MAR: 2 MBR: I2 IR: I2 ACC: 10 AL: None BL: None
PC: 3 MAR: 12 MBR: 314 IR: I2 ACC: 10 AL: 314 BL: None
PC: 4 MAR: 3 MBR: I3 IR: I3 ACC: 10 AL: 314 BL: None
PC: 4 MAR: 5 MBR: 123 IR: I3 ACC: 10 AL: 314 BL: None
PC: 4 MAR: 123 MBR: 123 IR: I3 ACC: 10 AL: 314 BL: None
PC: 5 MAR: 4 MBR: I4 IR: I4 ACC: 10 AL: 314 BL: None
